In [21]:
# Azure AI Search Initialization
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the service name and admin key from environment variables
service_name = os.getenv('AZURE_SERVICE_NAME')
admin_key = os.getenv('AZURE_ADMIN_KEY')

# Use the service name and admin key as before
endpoint = f"https://{service_name}.search.windows.net"
credential = AzureKeyCredential(admin_key)



In [24]:
# Create an index
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex
)
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

# Define the index schema
index_name = "example-index"
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    SimpleField(name="url", type=SearchFieldDataType.String, searchable=False)
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)

# Define the semantic configuration
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)

print(f"Index '{index_name}' created successfully with vector search and semantic search configurations.")

Index 'example-index' created successfully with vector search and semantic search configurations.


In [20]:
# Upload documents to the index
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Upload documents to the index
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Define the documents to upload
documents = [
    {
        "id": "1",
        "title": "Sample Document 1",
        "content": "This is the content of the first sample document.",
        "contentVector": [0.1, 0.2, 0.3, 0.1536],  # Example vector with 1536 dimensions
        "url": "http://example.com/doc1"
    },
    {
        "id": "2",
        "title": "Sample Document 2",
        "content": "This is the content of the second sample document.",
        "contentVector": [0.2, 0.3, 0.4, 0.1536],  # Example vector with 1536 dimensions
        "url": "http://example.com/doc2"
    }
]

# Upload the documents
result = search_client.upload_documents(documents=documents)

print(f"Documents uploaded: {result}")


HttpResponseError: (OperationNotAllowed) The request is invalid. Details: actions : 0: There's a mismatch in vector dimensions. The vector field 'contentVector', with dimension of '1536', expects a length of '1536'. However, the provided vector has a length of '4'. Please ensure that the vector length matches the expected length of the vector field. Read the following documentation for more details: https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-configure-compression-storage. 1: There's a mismatch in vector dimensions. The vector field 'contentVector', with dimension of '1536', expects a length of '1536'. However, the provided vector has a length of '4'. Please ensure that the vector length matches the expected length of the vector field. Read the following documentation for more details: https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-configure-compression-storage.
Code: OperationNotAllowed
Message: The request is invalid. Details: actions : 0:

In [23]:
# Delete the index
index_client.delete_index(index_name)

print(f"Index '{index_name}' deleted successfully.")

Index 'example-index' deleted successfully.
